# IGVF MPRA workshop MPRAsnakeflow tutorial

In this tutorial we will go over the results of MPRAsnakeflow on the example data provided [in this repository](https://github.com/kircherlab/MPRAsnakeflow_tutorial/tree/main)
It is not necessary to run MPRAsnakeflow in advance as the resulting files are shared in the Drive.

To start working on this tutorial in Colab, save a copy on your Google Drive (`File` $\rightarrow$ `Save a copy in Drive`).

To work on the tutorial locally, you can download this assignment as an IPython Notebook (`File` $\rightarrow$ `Download` $\rightarrow$ `Download .ipynb`). To work locally, you need to have Jupyter installed (ideally in a conda environment as discribed [here](https://test-jupyter.readthedocs.io/en/latest/install.html#id3)).

In [ ]:
import matplotlib.pyplot as plt
import os
import cv2
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [ ]:
data_dir = "/content/gdrive/MyDrive/MPRA-IGVF-Workshop-2024/Tutorials/T2_MPRA_stats/MPRAsnakeflow_data"

First, let's take a look at the structure of our input files:

In [ ]:
!ls -R "{data_dir}"

/content/gdrive/MyDrive/MPRA-IGVF-Workshop-2024/Tutorials/T2_MPRA_stats/MPRAsnakeflow_data:
assignment  counts  design  experiment.csv  workshop_config.yaml

/content/gdrive/MyDrive/MPRA-IGVF-Workshop-2024/Tutorials/T2_MPRA_stats/MPRAsnakeflow_data/assignment:
assoc_bc.fastq.gz  assoc_fwd.fastq.gz  assoc_rev.fastq.gz

/content/gdrive/MyDrive/MPRA-IGVF-Workshop-2024/Tutorials/T2_MPRA_stats/MPRAsnakeflow_data/counts:
dna-1_S1_R1_001.fastq.gz  dna-3_S3_R2_001.fastq.gz  rna-2_S5_R2_001.fastq.gz
dna-1_S1_R2_001.fastq.gz  dna-3_S3_R3_001.fastq.gz  rna-2_S5_R3_001.fastq.gz
dna-1_S1_R3_001.fastq.gz  rna-1_S4_R1_001.fastq.gz  rna-3_S6_R1_001.fastq.gz
dna-2_S2_R1_001.fastq.gz  rna-1_S4_R2_001.fastq.gz  rna-3_S6_R2_001.fastq.gz
dna-2_S2_R2_001.fastq.gz  rna-1_S4_R3_001.fastq.gz  rna-3_S6_R3_001.fastq.gz
dna-2_S2_R3_001.fastq.gz  rna-1_S6_R3_001.fastq.gz
dna-3_S3_R1_001.fastq.gz  rna-2_S5_R1_001.fastq.gz

/content/gdrive/MyDrive/MPRA-IGVF-Workshop-2024/Tutorials/T2_MPRA_stats/MPRAsnakeflow_data/de

----
## Assignment step (optional)

To eventually analyze your MPRA data, you need to know which barcode goes with which tested sequence. This is called assignment and typically associated multiple barcodes (i.e. the sequences that you will later count in the MPRA experiment) to a target sequence. It is possible that you already designed your assignment in advance (e.g. have barcodes synthezied with the targeted sequences) and already have a file which holds for each of the designed sequences the assignment to the barcodes. If this is the case you do not need to run the assignment step again and can run the experiment workflow with the file you already have.\
For the assignment step, we will need the files in the assignment folder (sequencing of barcodes and of the target sequences), as well as the config file and the design fasta (target sequences) in the design folder. We will take a look at these files now. The structure of a fastq file is as follows:  


```
@Sequence ID 1  
Sequence 1  
+(Sequence ID 1)  
Quality score string of Sequence 1  
@Sequence ID 2  
Sequence 2  
+(Sequence ID 2)
Quality score string of sequence 2  
...
```


In [1]:
print("First two barcodes in our barcode sequencing file:\n")
!zcat "{data_dir}/assignment/assoc_bc.fastq.gz" | head -n 8

print("\nFirst two sequences in our forward sequencing file:\n")
!zcat "{data_dir}/assignment/assoc_fwd.fastq.gz" | head -n 8

First two barcodes in our barcode sequencing file:

gzip: {data_dir}/assignment/assoc_bc.fastq.gz: No such file or directory

First two sequences in our forward sequencing file:

gzip: {data_dir}/assignment/assoc_fwd.fastq.gz: No such file or directory


If you take a good look at these files, you can see the sequence IDs in lines starting with the "@". By definition the sequence ID ends at the first white space (remainder is considered a comment). This sequence ID is identical between the different files, i.e. the forward, reverse and barcode reads from the same molecule are in the same order. From the combination of these files, we can see which barcode is associated with which sequenced target (forward/reverse reads). These sequenced forward/reverse reads should therefore correspond to the sequences that we designed. The designed sequences can be found in our design fasta, which contains the following information:  

```
>oligo ID 1  
Designed sequence 1
>oligo ID 2  
Designed sequence 2
...
```




In [ ]:
!head "{data_dir}/design/workshop_design.fa"

"Oligos" are the target sequences that we designed in advance, i.e. the sequences we would like to see in our assignment sequencing files (fastq files). In the association step, our goal is to assign the sequences to the oligos, since as you can see the oligo ID and sequence ID are not matching. By matching the reads to our designed oligos, we also automatically link the sequenced barcodes to our designed oligos, which is essential for us later.

Let's take a look at the part of the config file that concerns the assignment part.

In [ ]:
!sed '/experiments/Q' "{data_dir}/workshop_config.yaml"

As you can see this part of the config has three different positions for fastq files. Namely, `FW`, `BC` and `REV`, the abbreviations for forward, barcode and reverse reads. Thus, the corresponding files should be added here.



As a sanity check and to understand where these values come from, we will take a look at if the information matches our input files. We used 15 bp adapters for this MPRA experiment, which were added at both ends of each designed oligo. The ``alignment_start`` configuration refers to these adapters as our actual target sequences only starts after the 15 bp of adapter sequence.

In [ ]:
adapter_length=15

print("\nAverage length of a barcode:")
!zcat "{data_dir}/assignment/assoc_bc.fastq.gz" | awk 'NR % 4 == 2 {{sum += length($$0)}} END {{print sum*4/NR}}'

print("\nAverage length of a target sequence (oligo length - 2 x adapter):")
seq_length_with_bc = !cat "{data_dir}/design/workshop_design.fa" | awk 'NR % 4 == 2 {{sum +=length($$0)}} END {{print sum*4/NR}}'
seq_length_with_bc = int(seq_length_with_bc[0])
seq_length = seq_length_with_bc - 2*adapter_length
print(seq_length)

Finally, the ``min_support`` and ``fraction`` are necessary to resolve what to do in the situation where the same barcode sequence is observed with multiple oligos. In this scenario and using the given ``standardConfig`` setting, the barcode will be assigned to its most frequently observed oligo if the barcode was observed at least three times and if the most frequent oligo explained at least 70% of the occurances of this barcode.

### Assignment step results

Since running MPRAsnakeflow takes longer than the time scheduled for this tutorial, we uploaded some of the relevant output files to this drive. Just for the curious people among you, for the example data the pipeline takes 1h using just one core for both tasks, assignment and experiment. The results folder structure should be the same as after running the assignment step using our example config file (explained above).

In [ ]:
# Let's take a look at the files in the assignment results folder:
results_dir = "/content/gdrive/MyDrive/MPRA-IGVF-Workshop-2024/Tutorials/T2_MPRA_stats/MPRAsnakeflow_results/assignment"
!ls -R "{results_dir}"


The statistic folder contains run statistics for the assignment step, such as the total number of associated barcounts, the number of oligos with at least 15 barcodes, and more. We will now just take a look at the generated count histogram of barcodes per oligo.

In [ ]:
img = plt.imread(os.path.join(results_dir, "assignMPRAworkshop/statistic/assignment.standardConfig.png"))
plt.imshow(img)
plt.axis("off")

The median number of barcodes is 24. We usually require each tested sequence to have at least 10 barcodes associated.

We will now take a look at the actual output file of the assignment run. Which is a sorted table of `barcodes`, their assigned `sequence IDs`, information about the `quality` of the alignment (fwd/rev reads to designed oligos) and how often this barcode was assigned to this sequence compared to the total number this barcode was observed in the experiment. The information in the quality field is derived from the SAM/BAM output (https://samtools.github.io/hts-specs/SAMv1.pdf) and represents a concatenation of the SAM position, cigar, NM, MD and mapQ.

In [ ]:
assignment = os.path.join(results_dir, "assignMPRAworkshop/assignment_barcodes.standardConfig.sorted.tsv.gz")
!zcat {assignment} | head

We would like to know how many different oligos have been included in the final assignment.\
When we compare this number to the number of sequences in our design, we can get a feeling of how many sequences were able to get assigned with their barcodes. This has implications for the proportion of targets that we can eventually analyse and whether each target is supported with sufficiently many barcodes. This information is important for the experiment overall and analyzing the counts of each individual replicate.

In [ ]:
assigned_sequences = !zcat /content/gdrive/MyDrive/MPRA-IGVF-Workshop-2024/Tutorials/T2_MPRA_stats/MPRAsnakeflow_results/assignment/assignMPRAworkshop/assignment_barcodes.standardConfig.sorted.tsv.gz | awk -F'\t' '{names[$2]++} END {print length(names)}'

assigned_sequences = int(assigned_sequences[0])
print("Number of sequences we were able to assign: %s"%(assigned_sequences))

designed_sequences = !cat "{data_dir}/design/workshop_design.fa" | grep -v ">" | wc -l
designed_sequences = int(designed_sequences[0])
print("Number of sequences in the initial design: %s"%(designed_sequences))

print(r"Conclusion: %s%% sequences of our initial design can be used in the experimental/counting step"%(round(assigned_sequences/designed_sequences * 100, 1)))

-----
## Experiment step

For the experiment step, where the counts per oligo are calculated, we need all the files in the MPRAsnakeflow_data/counts folder, as well as the config file and the barcode assignment file, which is the output of the assignment step.

We investigated the assignment file in the end of the previous section. This table is now used to count the number of observed barcodes for each target sequence.

First we will cover the config file and afterwards we will look at interesting results like the correlation of the DNA and RNA counts among the replicates. This is useful in practice to check if the data preparation went well, because if the resulting data is not correlating between replicates, problems during the experiment will introduce a bias in the results of our analysis that we would like to know about.

The following block shows the part of the config file that concerns the experiment part. Since assignment and experiment part can be run independently the `bc_length` needs to be set again.

The `experiment_file` holds the path to the file of the experiment table (i.e. condition, replicate and name of read files given).

In [ ]:
!sed -n '/experiments:/,$p' "{data_dir}/workshop_config.yaml"

We can once again take a look at the input files, to understand where these cnofigurations come from. Each count experiment, in our case, is described using three files (the paths of which are in the experiment.csv file): forward reads, reverse reads, and UMIs. Note that the UMIs are optional, and serve to deduplicate barcodes with overdispersed counts due to preferential amplification related to GC content or library preparation steps. From one of the UMI read files, we can see that indeed our UMI length is 16:

In [ ]:
print("\nAverage length of a UMI:")
!zcat "{data_dir}/counts/dna-1_S1_R2_001.fastq.gz" | awk 'NR % 4 == 2 {{sum += length($$0)}} END {{print sum*4/NR}}'

We can take a look at the amount of barcodes that have been removed due to UMI collision, by checking how many unique BC x UMI combinations we have in our data. (We are now checking the RNA counts in replicate 1, but we should look at both DNA and RNA counts in all three replicates.)

In [ ]:
results_dir = "/content/gdrive/MyDrive/MPRA-IGVF-Workshop-2024/Tutorials/T2_MPRA_stats/MPRAsnakeflow_results/experiments"
print("First few lines of a count file, where column 1 contains the barcode, 2 the UMI and 3 the count.")
!zcat "{results_dir}"/workshop_data/counts/HepG2_1_RNA_filtered_counts.tsv.gz | head

print("\nAmount of total BC x UMI combinations:")
!zcat "{results_dir}"/workshop_data/counts/HepG2_1_RNA_filtered_counts.tsv.gz | wc -l

print("\nAmount of duplicate BC x UMI combinations (the file is sorted):")
!zcat "{results_dir}"/workshop_data/counts/HepG2_1_RNA_filtered_counts.tsv.gz | uniq -d | wc -l

There are no collisions, indicating that this subset of the data is of high quality. Great!

The count files are then joined with the assignment file (the output of the assignment step) to assign counts to oligos. The barcode counts are then aggregated over the oligos. This is done within MPRAsnakeflow with additional filter steps, but to give you a feeling of what's happening we give you some simplified code below:

In [ ]:
counts_rep1 = os.path.join(results_dir, "workshop_data/assigned_counts/MPRAworkshop/HepG2_1.merged.config.tutorialConfig.tsv.gz")
print("Joining the assignment and the counts of replicate 1:\n\nBarcode\toligo\tDNA\tRNA")
!join <(zcat {assignment}) <(zcat {counts_rep1}) -t $'\t' | cut -f 1,2,5-6 | head

print("\nAggregating the barcodes over oligos and the counts of replicate 1:\n\noligo\tDNA\tRNA")
!join <(zcat {assignment}) <(zcat {counts_rep1}) -t $'\t' | cut -f 1,2,5-6 | sort -k2,2 | \
 awk -v OFS="\t" '{{if ($$2 == prev) {{dna += $$3; rna += $$4}} else {{ if (NR > 1) print prev,dna,rna; prev=$$2; dna=$$3;rna=$$4}}}}' | head

In the next steps, barcodes are already aggregated to their oligos/targets.  
  
We will have a look at the correlation plots between the replicates for DNA and RNA now. Typically, we will sequence RNA counts deeper than DNA -- because of the larger dynamic range of RNA. We therefore expect that the correlation of RNA counts is higher than that of the DNA. Our example data shows this trend as well.

We have a correlation of DNA counts per insert between the replicates of 0.27 and 0.31. This correlation is for a subset of the complete MPRA experiment. The shown output uses the user set minThreshold for removing barcodes with too few counts. In order to see the influence of the parameter another plot is generated without this threshold.

With the threshold of 5 barcodes per oligo and at least one DNA and RNA count
- DNA correlation: 0.28 - 0.31
- RNA correlation: 0.68 - 0.70

In [ ]:
dna_pairwise = cv2.imread(f'{results_dir}/workshop_data/statistic/assigned_counts/MPRAworkshop/tutorialConfig/HepG2_DNA_pairwise_minThreshold.png')

dna_pairwise_resize = cv2.resize(dna_pairwise, (600, 1000))

cv2_imshow(dna_pairwise_resize)

In [ ]:
rna_pairwise = cv2.imread(f'{results_dir}/workshop_data/statistic/assigned_counts/MPRAworkshop/tutorialConfig/HepG2_RNA_pairwise_minThreshold.png')

rna_pairwise_resize = cv2.resize(rna_pairwise, (600, 1000))

cv2_imshow(rna_pairwise_resize)

And without these thresholds:
- DNA correlation: 0.25 - 0.27
- RNA correlation: 0.58 - 0.61

In [ ]:
dna_pairwise_no_thresh = cv2.imread(f'{results_dir}/workshop_data/statistic/assigned_counts/MPRAworkshop/tutorialConfig/HepG2_DNA_pairwise.png')

dna_pairwise_no_thresh_resize = cv2.resize(dna_pairwise_no_thresh, (600, 1000))

cv2_imshow(dna_pairwise_no_thresh_resize)

In [ ]:
rna_pairwise_no_thresh = cv2.imread(f'{results_dir}/workshop_data/statistic/assigned_counts/MPRAworkshop/tutorialConfig/HepG2_RNA_pairwise.png')

rna_pairwise_no_thresh_resize = cv2.resize(rna_pairwise_no_thresh, (600, 1000))

cv2_imshow(rna_pairwise_no_thresh_resize)

In the following two figures you can see that this trend is continues for a threshold of 10 barcodes per insert.
- DNA correlation: 0.31 - 0.39
- RNA correlation: 0.79 - 0.80

In [ ]:
dna_pairwise_no_thresh = cv2.imread(f'{results_dir}/workshop_data/statistic/assigned_counts/MPRAworkshop/tutorialConfig_10bc/HepG2_DNA_pairwise_minThreshold.png')

dna_pairwise_no_thresh_resize = cv2.resize(dna_pairwise_no_thresh, (600, 1000))

cv2_imshow(dna_pairwise_no_thresh_resize)

In [ ]:
rna_pairwise_no_thresh = cv2.imread(f'{results_dir}/workshop_data/statistic/assigned_counts/MPRAworkshop/tutorialConfig_10bc/HepG2_RNA_pairwise_minThreshold.png')

rna_pairwise_no_thresh_resize = cv2.resize(rna_pairwise_no_thresh, (600, 1000))

cv2_imshow(rna_pairwise_no_thresh_resize)

Even though it is a small example, the minimal barcode count threshold shows a clear effect on correlation. However, we also note that we are losing a proprotion of oligos with these strict filters and thereby limit the conclusions that we can draw from the data.

Now we want go through the final output of MPRAsnakeflow: the count table of sequences and barcodes together with their aggregated log2 values. Note that there are no statistics calculated at this step yet, the table doesn't say anything about certainty/variance of the activity. The log2 value refers to the log2 ratio of RNA counts over DNA counts for each oligo, which is a measurement of the activity of the target sequence. More on this in the statistical analysis tutorial.

In [ ]:
 !zcat {results_dir}/workshop_data/assigned_counts/MPRAworkshop/tutorialConfig/HepG2_allreps_merged.tsv.gz | head

This is also the file that you will be using in the next 45 minutes. Don't hesitate to ask questions, and we hope you enjoyed this small introduction to MPRAsnakeflow.